In [19]:

import numpy as np
import networkx as nx
import itertools
import gudhi as gd
###########################################################examples
#G = nx.circulant_graph(7,[1])
#simplices = nx.enumerate_all_cliques(G)
#simplices = [i for i in simplices]
#simplices = [[0], [1], [2], [3]]

#simplices = [[0],[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[0,1],[0,2],[1,2],[2,3],[3,4],[3,5],[3,6],[4,5],[4,6],[5,6],[6,7],[7,8],[7,9],[7,10],[8,9],[8,10],[9,10],[3,4,5],[3,4,6],[3,5,6],[4,5,6],[7,8,9],[7,8,10],[7,9,10],[8,9,10]]
#Here is our triangulation:
#   0-----3-----4-----0
#   | \   | \   | \   | \   |
#   |   \ |   \ |    \|   \ | 
#   1-----8-----7-----1
#   | \   | \   | \   | \   |
#   |   \ |   \ |   \ |   \ |
#   2-----5-----6-----2
#   | \   | \   | \   | \   |
#   |   \ |   \ |   \ |   \ |
#   0-----3-----4-----0
st = gd.SimplexTree()
st.insert([0,1,8])
st.insert([0,3,8])
st.insert([3,7,8])
st.insert([3,4,7])
st.insert([1,4,7])
st.insert([0,1,4])
st.insert([1,2,5])
st.insert([1,5,8])
st.insert([5,6,8])
st.insert([6,7,8])
st.insert([2,6,7])
st.insert([1,2,7])
st.insert([0,2,3])
st.insert([2,3,5])
st.insert([3,4,5])
st.insert([4,5,6])
st.insert([0,4,6])
st.insert([0,2,6])

simplices = [a for a,b in st.get_simplices()]
simplices = sorted(simplices)
#print(simplices)



In [20]:
#functions
def k_th_simplices(k,simplicial_complex):
    k_simplexes = [i for i in simplicial_complex if len(i)==k]
    return k_simplexes

def face(simplex):
    return [list(i) for i in list(itertools.combinations(simplex, len(simplex)-1))]

#define a function that returns true or false if sigma is face of tau(automatically just checks for dim(sigma) = dim(tau)-1 simplices)
def adj(sigma,tau):
    faces = face(tau)
    if sigma in faces:
        return True
    else:
        return False
#reemove an element in a list of a list type of structure
def remove(input_list,element):
    input_list = np.array(input_list)
    for i in element:
        input_list = np.delete(input_list,input_list.tolist().index(i),axis = 0)
    return input_list.tolist()
#checks if an unmatched edge is adjecent to only 1 unmatched triangle or not

#def check(triangle_list,edge):
    
    

#get all the k+1 simplices that have k simpex as their face

nodes = k_th_simplices(1,simplices)
edges = k_th_simplices(2,simplices)
triangles = k_th_simplices(3,simplices)



print(triangles)


[[0, 1, 4], [0, 1, 8], [0, 2, 3], [0, 2, 6], [0, 3, 8], [0, 4, 6], [1, 2, 5], [1, 2, 7], [1, 4, 7], [1, 5, 8], [2, 3, 5], [2, 6, 7], [3, 4, 5], [3, 4, 7], [3, 7, 8], [4, 5, 6], [5, 6, 8], [6, 7, 8]]


In [21]:
#NODES TO EDGES MATCHING

#consider a node to be critical

lc = nodes[6]
acyclic = []
matched_edges = []
matched_nodes = []
#get all the edges that have 0 node as its faces
for i in edges:
    if adj(lc,i) == True:
        acyclic.append((remove(i,lc),i))
        matched_edges.append(i)
        matched_nodes.append(remove(i,lc))

for j in matched_nodes:
    lc = j
    for i in remove(edges,matched_edges):
        if adj(j,i) == True and remove(i,j) not in matched_nodes:
            acyclic.append((remove(i,lc),i))
            matched_edges.append(i)
            matched_nodes.append(remove(i,lc))
        else:
            continue


unmatched_nodes = [i for i in remove(nodes,matched_nodes)]
unmatched_edges = [i for i in remove(edges,matched_edges)]
AC_critical_node = [i for i in unmatched_nodes]
print(len(edges))

27


In [22]:
matched_triangles = []
#############################################GENERALIZE THIS PART
#print(critical_edges)
#get all the triangles for 2 edges are matched
#map the unmatched edge into that triangle
AC_critical_edge = []
    
for i in triangles:
    for j in unmatched_edges:  
        if j in face(i):
            A = remove(face(i),[j]) 
            a,b = A[0],A[1] 
            if a in matched_edges and b in matched_edges and i not in matched_triangles and j not in matched_edges:
                matched_triangles.append(i)
                matched_edges.append(j)
                acyclic.append((j,i))

     

unmatched_triangles = remove(triangles,matched_triangles)
unmatched_edges = remove(edges,matched_edges)
temp = 0
while len(matched_edges)+len(AC_critical_edge)< len(edges):
    for i in triangles:
        for j in unmatched_edges:  
            if j in face(i)  and j not in AC_critical_edge and j not in matched_edges:
                A = remove(face(i),[j]) 
                a,b = A[0],A[1] 
                if a in matched_edges+AC_critical_edge and b in matched_edges+AC_critical_edge and i not in matched_triangles:
                    matched_triangles.append(i)
                    matched_edges.append(j)
                    acyclic.append((j,i))

    unmatched_edges = [i for i in remove(edges,matched_edges)]
    if len(AC_critical_edge)<len(unmatched_edges):
        AC_critical_edge.append(unmatched_edges[temp])
        temp = temp+1
    


AC_critical_triangle = remove(triangles,matched_triangles)
print(AC_critical_node)
print(AC_critical_edge)
print(AC_critical_triangle)
###############################################################################
#please note that here the algorithm where while trying to match the edges with all the nodes and all the triangles, 
# need to make sure that the edge is adjecent to only to 1 triangle i.e define  check fn such that edges match
#with triangles only if there are no other alternatives.

#in our above if condition I have used that both other edges must be matched for a triangle inorder for the triangle to be matched with
#the unmatched edges. The else condition where I need to force that one of the edges need to be made Critical is not added!!!!!!!!!

[[6]]
[[0, 8], [1, 2]]
[[3, 4, 7]]


In [23]:
#gradient paths
#################################GET THE GRADIENT PATHS FOR THE TRIANGLE AND EDGES
#print(acyclic)
#get a critical edge from the critical edge list
#get a corresponding critical node from the critical node list

def path_iter(c_edge,c_node,matched_edge_set): #input:critical edge,matched edge set
    gradient_paths = []
    
    for z in face(c_edge):
        g1 = []
        
        for j in range(len(matched_edge_set)):
            for i in matched_edge_set:
                if z != c_node and adj(z,i)==True and i not in g1:
                    g1.append(i)
                    z = remove(i,z)

        if c_node in face(g1[-1]):            
            gradient_paths.append(g1)
                   
    return gradient_paths

print(path_iter(AC_critical_edge[1],AC_critical_node[0],matched_edges))



#define a function where given an edge it gives another edge that share a same node 

[[[0, 1], [0, 3], [2, 3], [2, 5], [3, 5], [3, 4], [4, 6]], [[2, 6]]]


In [24]:
def path_iter_tri(critical_triangle,critical_edge,ACyclic_matched_list):
    gradient_paths = []
    non_gradient_paths = []
    temp = [b for a,b in ACyclic_matched_list if len(b)==3]
    print(temp)
    for z in face(critical_triangle):
        g1 = []
        
        for j in range(len(temp)):
            for i in temp:
                if z != critical_edge and adj(z,i)==True and i not in g1 and (z,i) in ACyclic_matched_list:
                    g1.append(i)
                    z = remove(i,z)
        if critical_edge in face(g1[-1]):            
            gradient_paths.append(g1)
                    
    return gradient_paths

print(path_iter_tri(AC_critical_triangle[0] ,AC_critical_edge[0],acyclic))

[[0, 2, 6], [0, 4, 6], [2, 6, 7], [4, 5, 6], [5, 6, 8], [6, 7, 8], [0, 1, 4], [0, 2, 3], [0, 1, 8], [0, 3, 8], [1, 5, 8], [3, 7, 8], [1, 2, 5], [1, 2, 7], [1, 4, 7], [2, 3, 5], [3, 4, 5]]
[]
